In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
%cd /content/drive/MyDrive/Autonomous-AI/Day1/1_07_CV_LineDetection/kmu_LineDetection/

/content/drive/MyDrive/Autonomous-AI/Day1/1_07_CV_LineDetection/kmu_LineDetection


In [ ]:
import cv2
import numpy as np
import os
from scipy import optimize
from matplotlib import pyplot as plt, cm, colors
from google.colab.patches import cv2_imshow

ym_per_pix = 30 / 720
xm_per_pix = 3.7 / 720

cap = cv2.VideoCapture('/content/drive/MyDrive/Autonomous-AI/Day1/1_07_CV_LineDetection/kmu_LineDetection/kmu_track.avi')

fourcc = cv2.VideoWriter_fourcc(*'DIVX') 
out = cv2.VideoWriter('/content/drive/MyDrive/Autonomous-AI/Day1/1_07_CV_LineDetection/kmu_LineDetection/kmutrack_output.avi', fourcc, 20.0, (640, 480))

def processImage(inpImage):
    inpImage = cv2.cvtColor(inpImage, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(inpImage, (5, 5), 0)
    canny = cv2.Canny(blur, 60, 200)

    return canny

def perspectiveWarp(inpImage):
    img_size = (inpImage.shape[1], inpImage.shape[0])

    src = np.float32([[100, 300],
                      [0, 460],
                      [640, 460],
                      [470, 300]])
    dst = np.float32([[150, 0],
                      [230, 455],
                      [500, 455],
                      [500, 0]])

    matrix = cv2.getPerspectiveTransform(src, dst)

    minv = cv2.getPerspectiveTransform(dst, src)
    birdseye = cv2.warpPerspective(inpImage, matrix, img_size)

    return birdseye, minv


def slide_window_search(binary_warped):
    out_img = np.dstack((binary_warped, binary_warped, binary_warped)) * 255
    window_img = np.zeros_like(out_img)
    midpoint = binary_warped.shape[1] // 2
    histogram = np.sum(binary_warped[binary_warped.shape[0] // 2:, :], axis=0)
    leftx_base = np.argmax(histogram[:3 * midpoint // 4])
    rightx_base = np.argmax(histogram[3 * midpoint // 2:]) + 3 * midpoint // 2
    for_mid_histogram = np.sum(binary_warped[:binary_warped.shape[0] // 2, :], axis=0)
    left_middle_base = np.argmax(for_mid_histogram[:midpoint])
    right_middle_base = np.argmax(for_mid_histogram[3 * midpoint // 2:]) + 3 * midpoint // 2

    nwindows = 30
    window_height = np.int(binary_warped.shape[0] / nwindows)
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    leftx_current = leftx_base
    rightx_current = rightx_base

    margin = 20
    minpix = 5
    minimal_window = 1
    left_lane_inds = []
    right_lane_inds = []

    left_cnt_window, right_cnt_window = 0, 0

    for window in range(nwindows):
        if window == nwindows // 2:
            leftx_current = leftx_current if left_cnt_window > minimal_window else left_middle_base
            rightx_current = rightx_current if right_cnt_window > minimal_window else right_middle_base

        win_y_low = binary_warped.shape[0] - (window + 1) * window_height
        win_y_high = binary_warped.shape[0] - window * window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        cv2.rectangle(out_img, (win_xleft_low, win_y_low), (win_xleft_high, win_y_high), (0, 255, 0), 2)
        cv2.rectangle(out_img, (win_xright_low, win_y_low), (win_xright_high, win_y_high), (0, 255, 0), 2)

        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (
                nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (
                nonzerox < win_xright_high)).nonzero()[0]

        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            left_cnt_window += 1
        else:
            left_idx = win_y_high

        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
            right_cnt_window += 1
        else:
            right_idx = win_y_high

    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    left_fit = [0, 0, 0]
    right_fit = [0, 0, binary_warped.shape[1]]

    if len(leftx) != 0 and len(lefty) != 0 and len(rightx) != 0 and len(righty) != 0:
        if rightx.mean() - leftx.mean() > 320:
            left_fit = np.polyfit(lefty, leftx, 2)
            right_fit = np.polyfit(righty, rightx, 2)
        else:
            if left_cnt_window > right_cnt_window:
                left_fit = np.polyfit(lefty, leftx, 2)
            else:
                right_fit = np.polyfit(righty, rightx, 2)

    elif len(leftx) == 0 or len(lefty) == 0:
        right_fit = np.polyfit(righty, rightx, 2)

    elif len(rightx) == 0 or len(righty) == 0:
        left_fit = np.polyfit(lefty, leftx, 2)

    ploty = np.linspace(0, binary_warped.shape[0] - 1, binary_warped.shape[0])

    left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]

    for j in range(binary_warped.shape[0]):
        if ploty[j] > left_idx:
            left_fitx[j] = 0

        if ploty[j] > right_idx:
            right_fitx[j] = 640

    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx - margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx + margin, ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx - margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx + margin, ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    cv2.fillPoly(window_img, np.int_([left_line_pts]), (255, 0, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0, 255, 0))

    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

    ret = {}
    ret['leftx'] = leftx
    ret['rightx'] = rightx
    ret['left_fitx'] = left_fitx
    ret['right_fitx'] = right_fitx
    ret['ploty'] = ploty

    return ploty, left_fit, right_fit, ret


def draw_lane_lines(original_image, warped_image, Minv, draw_info):
    leftx = draw_info['leftx']
    rightx = draw_info['rightx']
    left_fitx = draw_info['left_fitx']
    right_fitx = draw_info['right_fitx']
    ploty = draw_info['ploty']

    warp_zero = np.zeros_like(warped_image).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    mean_x = np.mean((left_fitx, right_fitx), axis=0)
    pts_mean = np.array([np.flipud(np.transpose(np.vstack([mean_x, ploty])))])

    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))

    newwarp = cv2.warpPerspective(color_warp, Minv, (original_image.shape[1], original_image.shape[0]))
    result = cv2.addWeighted(original_image, 1, newwarp, 0.3, 0)

    return pts_mean, result

if __name__ == '__main__':
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            try:
                canny = processImage(frame)
                birdView, minverse = perspectiveWarp(canny)
                ploty, lfit, rfit, draw_info = slide_window_search(birdView)
                meanPts, result = draw_lane_lines(frame, birdView, minverse, draw_info)

            except:
                continue

            out.write(result)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [ ]:
from google.colab import files
files.download('/content/drive/MyDrive/Autonomous-AI/Day1/1_07_CV_LineDetection/kmu_LineDetection/kmutrack_output.avi')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>